In [1]:
import pandas as pd
import mlflow
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline





In [2]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'

In [3]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment('green-taxi-duration')

<Experiment: artifact_location='s3://marcospaulo-mlops/airflow/1', creation_time=1686499792049, experiment_id='1', last_update_time=1686499792049, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical]= df[categorical].astype(str)
    
    return df

def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    
    dicts = df[categorical + numerical].to_dict(orient='records')
    
    return dicts

In [5]:
df_train = read_dataframe('../../data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('../../data/green_tripdata_2022-02.parquet')

In [6]:
target = 'duration'

y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=20, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    
    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )
    
    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)
    
    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)
    
    mlflow.sklearn.log_model(pipeline, artifact_path='model')
    


{'max_depth': 20, 'n_estimators': 20, 'min_samples_leaf': 10, 'random_state': 0} 6.110884769689751


e:\Estudos\Python\MLOps-Zoomcamp\03-orchestration\.venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [15]:
RUN_ID = '96d8337f55ed460e8531034e0e8356e0'

client = MlflowClient(MLFLOW_TRACKING_URI)

In [16]:
# mlflow.artifacts.download_artifacts(run_id=RUN_ID, artifact_path='model/model.pkl')
path = client.download_artifacts(run_id=RUN_ID, path='model/model.pkl')


C:\Users\MarcosPaulo\AppData\Local\Temp\ipykernel_12388\3971535463.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path='model/model.pkl')


In [17]:
pipeline.named_steps

{'dictvectorizer': DictVectorizer(),
 'randomforestregressor': RandomForestRegressor(max_depth=20, min_samples_leaf=10, n_estimators=20,
                       n_jobs=-1, random_state=0)}

In [18]:
with open(path, 'rb') as f_out:
    pip = pickle.load(f_out)

In [19]:
pip

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=10,
                                       n_estimators=20, n_jobs=-1,
                                       random_state=0))])